## Deploy Llava-Video with Qwen 7B Endpoint on SageMaker


#### LMI is shipping vLLM 0.4.2 with 0.28.0 containers, so technically we support all LLM that vLLM 0.4.2 support.

https://docs.djl.ai/master/docs/serving/serving/docs/lmi/user_guides/vllm_user_guide.html

Multi Modal Models support with 0.28.0 containers 

- LlaVA-NeXT (llava-hf/llava-v1.6-mistral-7b-hf, llava-hf/llava-v1.6-vicuna-7b-hf, etc.)
- Phi-3-Vision (microsoft/Phi-3-vision-128k-instruct, etc.)

### Setup

In this notebook, we will deploy a Llava 1.6 endpoint with DJLServing container image.

This notebook has been tested within SageMaker Studio Notebook Instance with pytorch environment. 

This notebook has been tested on Amazon SageMaker Notebook Instances with single GPU on ml.g5.2xlarge

In [5]:
# !pip install huggingface_hub transformers==4.41.0 --upgrade --quiet

In [6]:
# !pip install sagemaker boto3 --upgrade

In [1]:
import sagemaker
import jinja2
from sagemaker import image_uris
import boto3
import os
import time
import json
from pathlib import Path
import json
import base64

/home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/28/25 06:22:55] INFO     Found credentials from IAM Role:                                   ]8;id=877760;file:///home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=557021;file:///home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             PVRE-SSMOnboardingRole-HNmtTTtLyNAa                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ubuntu/.config/sagemaker/config.yaml


In [2]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

[02/28/25 06:23:00] INFO     Found credentials from IAM Role:                                   ]8;id=593702;file:///home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=648571;file:///home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             PVRE-SSMOnboardingRole-HNmtTTtLyNAa                                                   

[02/28/25 06:23:01] INFO     Found credentials from IAM Role:                                   ]8;id=503108;file:///home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=633813;file:///home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             PVRE-SSMOnboardingRole-HNmtTTtLyNAa                                                   

In [3]:
model_bucket = sess.default_bucket()  # bucket to house artifacts
s3_code_prefix = "llava-video-code"  # folder within bucket where code artifact will go
s3_model_prefix = "models/LLaVA-Video-7B-Qwen2"  # folder within bucket where code artifact will go
region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

jinja_env = jinja2.Environment()

# define a variable to contain the s3url of the location that has the model
pretrained_model_location = f"s3://{model_bucket}/{s3_model_prefix}/"
print(f"Pretrained model will be uploaded to ---- > {pretrained_model_location}")

[02/28/25 06:23:04] INFO     Found credentials from IAM Role:                                   ]8;id=134048;file:///home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=135122;file:///home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             PVRE-SSMOnboardingRole-HNmtTTtLyNAa                                                   

Pretrained model will be uploaded to ---- > s3://sagemaker-us-west-2-452145973879/models/LLaVA-Video-7B-Qwen2/


## Prepare inference script and container image

Starting LMI V10 (0.28.0), we are changing the name from LMI DeepSpeed DLC to LMI (LargeModelInference). 

https://github.com/aws/deep-learning-containers/blob/master/available_images.md

In [5]:
# get the lmi image uri
# available frameworks: "djl-lmi" (for vllm, lmi-dist), "djl-tensorrtllm" (for tensorrt-llm), "djl-neuronx" (for transformers neuronx)

inference_image_uri = image_uris.retrieve(
    framework="djl-lmi", region=sess.boto_session.region_name, version="0.30.0"
)
inference_image_uri

[02/28/25 06:23:18] INFO     Ignoring unnecessary instance type: None.                            ]8;id=293483;file:///home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=584252;file:///home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

'763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.30.0-lmi12.0.0-cu124'

In this notebook, we will provide two ways to load the model when deploying to an endpoint.
- Directly load from Hugging Face 
- Store the model artifacts on S3 and load the model directly from S3

The [Large Model Inference (LMI)](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-dlc.html) container uses [s5cmd](https://github.com/peak/s5cmd) to download data from S3 which significantly reduces the speed when loading model during deployment. Therefore, we recommend to load the model from S3 by following the below section to download the model from Hugging Face and upload the model on S3. 

If you choose to load the model directly from Hugging Face during model deployment, you can skip the below section and jump to the section to **prepare the model tarbal file and upload to S3**.

### [OPTIONAL] Download the model from Hugging Face and upload the model artifacts on Amazon S3
If you intend to download your copy of the model and upload it to a s3 location in your AWS account, please follow the below steps, else you can skip to the next step.

In [11]:
from huggingface_hub import snapshot_download
from pathlib import Path

model_name = "lmms-lab/LLaVA-Video-7B-Qwen2"

# - This will download the model into the current directory where ever the jupyter notebook is running
local_model_path = Path("./llava-model")
local_model_path.mkdir(exist_ok=True)
# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model", "*.safetensors"]

# - Leverage the snapshot library to donload the model since the model is stored in repository using LFS
model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_model_path,
    allow_patterns=allow_patterns,
)

/home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 16 files: 100%|██████████| 16/16 [00:30<00:00,  1.90s/it]


In [12]:
model_download_path

'llava-model/models--lmms-lab--LLaVA-Video-7B-Qwen2/snapshots/013210b3aff822f1558b166d39c1046dd109520f'

Please make sure the file is downloaded correctly by checking the files exist in the newly created folder `llava-model/models--llava-hf--llava-v1.6-mistral-7b-hf` before running the below cell.

In [13]:
# upload the model artifacts to s3
model_artifact = sess.upload_data(path=model_download_path, key_prefix=s3_model_prefix)
print(f"Model uploaded to --- > {model_artifact}")
print(f"We will set option.s3url={model_artifact}")

Model uploaded to --- > s3://sagemaker-us-west-2-452145973879/models/LLaVA-Video-7B-Qwen2
We will set option.s3url=s3://sagemaker-us-west-2-452145973879/models/LLaVA-Video-7B-Qwen2


In [14]:
!rm -rf {local_model_path}

SageMaker Large Model Inference containers can be used to host models without providing your own inference code. This is extremely useful when there is no custom pre-processing of the input data or post-processing of the model's predictions.

SageMaker needs the model artifacts to be in a Tarball format. In this example, we provide the following files - `serving.properties` and `requirements.txt`.
- `serving.properties` is the configuration file that can be used to indicate to DJL Serving which model parallelization and inference optimization libraries you would like to use. Depending on your need, you can set the appropriate configuration. For more details on the configuration options and an exhaustive list, you can refer the [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html).
- `requirements.txt` is the text file containing any additional pip wheel need to install. 

If you want to download the model from huggingface.co, you can set option.model_id. The model id of a pretrained model hosted inside a model repository on huggingface.co (https://huggingface.co/models). The container uses this model id to download the corresponding model repository on huggingface.co. If you set the model_id to a s3 url, the DJL will download the model artifacts from s3 and swap the model_id to the actual location of the model artifacts. In your script, you can point to this value to load the pre-trained model.
- `option.tensor_parallel_degree`: Set to the number of GPU devices over which the model needs to be partitioned. This parameter also controls the number of workers per model which will be started up when DJL serving runs. As an example if we have a 8 GPU machine, and we are creating 8 partitions then we will have 1 worker per model to serve the requests.

More options can be found here [Container and Model Configurations](https://docs.djl.ai/master/docs/serving/serving/docs/lmi/deployment_guide/configurations.html), such as specify option.trust_remote_code=true to load and use the model


In [6]:
!mkdir llava-djl-config

mkdir: cannot create directory ‘llava-djl-config’: File exists


In [7]:
%%writefile llava-djl-config/serving.properties
engine = Python
option.tensor_parallel_degree = 1
option.model_loading_timeout = 600
option.model_id = {{s3url}}

Overwriting llava-djl-config/serving.properties


In [8]:
# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
template = jinja_env.from_string(Path("llava-djl-config/serving.properties").open().read())
Path("llava-djl-config/serving.properties").open("w").write(
    template.render(s3url=pretrained_model_location)
)
!pygmentize llava-djl-config/serving.properties | cat -n

     1	engine = Python
     2	option.tensor_parallel_degree = 1
     3	option.model_loading_timeout = 600
     4	option.model_id = s3://sagemaker-us-west-2-452145973879/models/LLaVA-Video-7B-Qwen2/


## Prepare the model tarball file and upload to S3

In [9]:
%%sh
tar czvf model.tar.gz llava-djl-config/

llava-djl-config/
llava-djl-config/serving.properties


In [10]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-452145973879/llava-video-code/model.tar.gz


## Deploy model

In [11]:
from sagemaker.model import Model
from sagemaker.utils import name_from_base
from datetime import datetime

deploy_model_name = "llava-video-7b-qwen2-" + datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
print(deploy_model_name)

llava-video-7b-qwen2-2025-02-28-06-23-53


In [12]:
model = Model(
    image_uri=inference_image_uri,
    model_data=s3_code_artifact,
    role=role,
    name=deploy_model_name,
)

In [13]:
%%time
endpoint_name = "endpoint-" + deploy_model_name
model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge",
    endpoint_name=endpoint_name
)

[02/28/25 06:23:58] INFO     Creating model with name: llava-video-7b-qwen2-2025-02-28-06-23-53     ]8;id=431065;file:///home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=653339;file:///home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

                    INFO     Creating endpoint-config with name                                     ]8;id=405586;file:///home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=858683;file:///home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\
                             endpoint-llava-video-7b-qwen2-2025-02-28-06-23-53                                     

[02/28/25 06:23:59] INFO     Creating endpoint with name                                            ]8;id=389105;file:///home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=727727;file:///home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\
                             endpoint-llava-video-7b-qwen2-2025-02-28-06-23-53                                     

------------------------------------------------------------*

[02/28/25 06:54:35] ERROR    Please check the troubleshooting guide for common errors:              ]8;id=365888;file:///home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=382555;file:///home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/sagemaker/session.py#4734\4734]8;;\
                             https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-t                
                             roubleshooting.html#sagemaker-python-sdk-troubleshooting-create-endpoi                
                             nt                                                                                    

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/IPython/core/magics/e │
│ xecution.py:1355 in time                                                                         │
│                                                                                                  │
│   1352 │   │   │   │   # multi-line %%time case                                                  │
│   1353 │   │   │   │   if expr_val is not None:                                                  │
│   1354 │   │   │   │   │   code_2 = self.shell.compile(expr_val, source, 'eval')                 │
│ ❱ 1355 │   │   │   │   │   out = eval(code_2, glob, local_ns)                                    │
│   1356 │   │   │   except:                                                                       │
│   1357 │   │   │   │   self.shell.showtraceback()                                                │
│   1358 │   │   │   │   return                                                                    │
│ in <module>:2                                                                                    │
│                                                                                                  │
│ /home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/sagemaker/model.py:17 │
│ 84 in deploy                                                                                     │
│                                                                                                  │
│   1781 │   │   │   if is_explainer_enabled:                                                      │
│   1782 │   │   │   │   explainer_config_dict = explainer_config._to_request_dict()               │
│   1783 │   │   │                                                                                 │
│ ❱ 1784 │   │   │   self.sagemaker_session.endpoint_from_production_variants(                     │
│   1785 │   │   │   │   name=self.endpoint_name,                                                  │
│   1786 │   │   │   │   production_variants=[production_variant],                                 │
│   1787 │   │   │   │   tags=tags,                                                                │
│                                                                                                  │
│ /home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/sagemaker/session.py: │
│ 5892 in endpoint_from_production_variants                                                        │
│                                                                                                  │
│   5889 │   │   logger.info("Creating endpoint-config with name %s", name)                        │
│   5890 │   │   self.sagemaker_client.create_endpoint_config(**config_options)                    │
│   5891 │   │                                                                                     │
│ ❱ 5892 │   │   return self.create_endpoint(                                                      │
│   5893 │   │   │   endpoint_name=name,                                                           │
│   5894 │   │   │   config_name=name,                                                             │
│   5895 │   │   │   tags=endpoint_tags,                                                           │
│                                                                                                  │
│ /home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/sagemaker/session.py: │
│ 4737 in create_endpoint                                                                          │
│                                                                                                  │
│   4734 │   │   │   logger.error(                                                                 │
│   4735 │   │   │   │   "Please check the troubleshooting gu

In [ ]:
endpoint_name 

## Test Inference Endpoint

#### Chat Completions API Schema

https://docs.djl.ai/master/docs/serving/serving/docs/lmi/user_guides/chat_input_output_schema.html

In [129]:
from PIL import Image
import base64
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")
endpoint_name = model.endpoint_name
#endpoint_name = "endpoint-llava-v16-mistral-7b-hf2024-08-16-05-20-21"

In [130]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

def run_inference(endpoint_name, inputs):
    response = smr_client.invoke_endpoint(
        EndpointName=endpoint_name, Body=json.dumps(inputs)
    )
    return response["Body"].read().decode('utf-8')

In [131]:
test_image = "food.jpg"
raw_image = Image.open(test_image).convert('RGB')

### You can check out the food image and the generated description

In [ ]:
display(raw_image.resize((468, 388)))

### Instructed zero-shot vision-to-language detailed caption generation

In [133]:
base64_image = encode_image(test_image)

inputs = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What is the content of image?"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                }
            ]
        }
    ],
    "max_tokens":256
  }

In [ ]:
response = run_inference(endpoint_name, inputs)
print(response)

### Instructed Visual Question Answering (VQA)

In [135]:
base64_image = encode_image(test_image)

inputs = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Is it recommended for eating before sport?"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                }
            ]
        }
    ],
    "max_tokens":256
  }

In [ ]:
response = run_inference(endpoint_name, inputs)
print(response)

In [141]:
base64_image = encode_image(test_image)

inputs = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Can you list several descriptions for product marketing?"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                }
            ]
        }
    ],
    "max_tokens":512
  }

In [ ]:
response = run_inference(endpoint_name, inputs)
print(response)

## Clean up
Uncomment the below cell to delete the endpoint and model when you finish the experiment

In [ ]:
#sm_client.delete_model(ModelName=deploy_model_name)
#sm_client.delete_endpoint(EndpointName=endpoint_name)